In [75]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    MultilayerPerceptron,
    Parser,
    PerceptronActivationType,
    Simulator,
    PlanarSceneGraphVisualizer,
    ZeroOrderHold,
)
from value_iteration import *
from functools import partial
import pandas as pd
import altair as alt

In [52]:
Q = np.diag([0.1, 20, 1, 1])
R = np.array([.2])

def train_cart_pole(num_samples=10, epochs=10, target_state=[0., np.pi, 0., 0.]):

    # Create the plant
    builder = DiagramBuilder()
    plant, _ = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    Parser(plant).AddModelFromFile("cartpole.urdf")
    plant.Finalize()
    plant_context = plant.CreateDefaultContext()
    builder.Build()

    # Create the state grid
    x_states_cart = np.linspace(-2, 2, num_samples)
    theta_states_cart = np.linspace(0, 2 * np.pi, 50)
    x_dot_states_cart = np.linspace(-10, 10, num_samples)
    theta_dot_states_cart = np.linspace(-10, 10, num_samples)
    state_grid_cart = np.meshgrid(
        x_states_cart,
        theta_states_cart,
        x_dot_states_cart,
        theta_dot_states_cart,
        indexing="ij",
    )
    state_grid = np.vstack([s.flatten() for s in state_grid_cart])

    # zero cost state
    target_state = np.array(target_state).reshape(-1, 1)

    # A neural network for the cartpole
    mlp = MultilayerPerceptron(
        [False, True, False, False],
        [128, 128, 1],
        [
            PerceptronActivationType.kReLU,
            PerceptronActivationType.kReLU,
            PerceptronActivationType.kIdentity,
        ],
    )

    # train the neural network
    mlp_context, loss_over_time = ContinuousFittedValueIteration(
        plant,
        plant_context,
        mlp,
        partial(compute_state_cost, Q, target_state),
        compute_u_star,
        R,
        state_grid,
        time_step=0.01,
        discount_factor=0.9999,
        lr=1e-4,
        minibatch=64,
        epochs=epochs,
        optimization_steps_per_epoch=100,
        input_limits=None,
        target_state=target_state,
    )
    
    J = mlp.BatchOutput(mlp_context, state_grid)[0].reshape(num_samples, 50, num_samples, num_samples)

    return mlp, mlp_context, loss_over_time, state_grid_cart, J

mlp, mlp_context, loss_over_time, state_grid, J = train_cart_pole(epochs=300)

In [53]:
losses = pd.DataFrame({"epoch": range(len(loss_over_time)), "loss": loss_over_time})
alt.Chart(losses).mark_line().encode(
    x="epoch",
    y="loss",
)

alt.Chart(...)

In [101]:
theta = 25
theta_dot = 0

x_by_xd = state_grid[0][:, theta, :, theta_dot]
xd_by_x = state_grid[2][:, theta, :, theta_dot]

# Convert this grid to columnar data expected by Altair
source = pd.DataFrame(
    {"x": x_by_xd.ravel(), "ẋ": xd_by_x.ravel(), "J": J[:, theta, :, theta_dot].ravel()}
)

alt.Chart(source).mark_rect().encode(
    alt.X("x:O", axis=alt.Axis(format=".2")),
    alt.Y("ẋ:O", axis=alt.Axis(format=".2")),
    alt.Color("J:Q", scale=alt.Scale(scheme="turbo")),
).properties(
    title="Cost-to-go",
)


alt.Chart(...)

# Input controller

In [ ]:
# initialize controller and plant
diagram_builder = DiagramBuilder()

cart_plant_cl, cart_scene_graph_cl = AddMultibodyPlantSceneGraph(
    diagram_builder, time_step=0.0
)

Parser(cart_plant_cl).AddModelFromFile("cartpole.urdf")
cart_plant_cl.Finalize()
cart_plant_context_cl = cart_plant_cl.CreateDefaultContext()
cart_controller_sys = ContinuousFittedValueIterationPolicy(
    cart_plant_cl,
    mlp,
    mlp_context,
    R,
    compute_u_star,
)

cart_controller = diagram_builder.AddSystem(cart_controller_sys)
# we assume a zero-order hold between time steps
time_step = 0.01
zoh_cart = diagram_builder.AddSystem(ZeroOrderHold(time_step, 1))

# wire all the systems together
diagram_builder.Connect(
    cart_plant_cl.get_state_output_port(), cart_controller.get_input_port()
)
diagram_builder.Connect(cart_controller.get_output_port(), zoh_cart.get_input_port())
diagram_builder.Connect(
    zoh_cart.get_output_port(), cart_plant_cl.get_actuation_input_port()
)

# Add visualizer
visualizer = diagram_builder.AddSystem(
    PlanarSceneGraphVisualizer(
        cart_scene_graph_cl, xlim=[-3.0, 3.0], ylim=[-.5, 1.2], show=False
    )
)
diagram_builder.Connect(
    cart_scene_graph_cl.get_query_output_port(), visualizer.get_input_port(0)
)

# Simulate
simulator = Simulator(diagram_builder.Build())
simulator.set_publish_every_time_step(False)  # makes sim faster

simulate_and_animate([0, np.pi, 0, 0], visualizer, simulator, sim_time=10)


# Test with fixed input controller

In [ ]:
def simulate_fixed_input_controller():
    # initialize plant
    diagram_builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(diagram_builder, time_step=0.0)
    Parser(plant).AddModelFromFile("cartpole.urdf")
    plant.Finalize()

    # Add visualizer
    visualizer = diagram_builder.AddSystem(
        PlanarSceneGraphVisualizer(
            scene_graph, xlim=[-3.0, 3.0], ylim=[-0.5, 1.2], show=False
        )
    )
    diagram_builder.Connect(
        scene_graph.get_query_output_port(), visualizer.get_input_port(0)
    )

    # set up simulator
    diagram = diagram_builder.Build()
    simulator = Simulator(diagram)
    simulator_context = simulator.get_mutable_context()

    # setup input
    fixed_input = np.array([10])
    plant_context = diagram.GetMutableSubsystemContext(plant, simulator_context)
    plant.get_actuation_input_port().FixValue(plant_context, fixed_input)

    simulate_and_animate([-2, np.pi, 0, 0], visualizer, simulator, sim_time=4)

simulate_fixed_input_controller()